# Build a simple ML pipeline with spark component

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with spark component

**Motivations** - In this example, we will explains how to create a spark component and use it in a pipeline. A Spark Component is a Component that executes a spark job in AML. It will support attached Synapse Spark pool and Managed (Automatic) Spark compute.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://learn.microsoft.com/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.2 Prepare spark workspace and compute resource
1. **Create an Azure Synapse workspace**, check [this](https://learn.microsoft.com/azure/synapse-analytics/quickstart-create-workspace) for more information.
2. **Create compute resource**, you can select from following two options:

    - Submit a Spark Job using Managed (Automatic) Spark compute, check [this](https://learn.microsoft.com/azure/machine-learning/interactive-data-wrangling-with-apache-spark-azure-ml#create-and-configure-managed-automatic-spark-compute-in-azure-machine-learning-notebooks) for more information about creating Managed (Automatic) Spark compute.
    
    - Submit a Spark Job using an attached Synapse Spark pool, check [this](https://learn.microsoft.com/azure/machine-learning/how-to-manage-synapse-spark-pool) for more information about attaching a Synapse Spark pool in Azure Machine Learning workspace.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, Output, load_component, command
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes

## 1.2 Configure credential
We are using `DefaultAzureCredential` to get access to workspace.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://learn.microsoft.com/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Synapse Spark pool.
spark_compute_target = "<ATTACHED_SPARK_POOL_NAME>"
print(ml_client.compute.get(spark_compute_target))

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Define components

Use `load_component` to load spark components defined using YAML. 

In [ ]:
# load component
parent_dir = "."
spark_kmeans = load_component(source=parent_dir + "/components/spark_kmeans.yml")

show_output_component = command(
    inputs=dict(spark_output=Input(type=AssetTypes.URI_FOLDER)),
    command="ls ${{inputs.spark_output}}",
    environment="azureml://registries/azureml/environments/sklearn-1.5/labels/latest",
)

# 3. Build pipeline

We define a pipeline containing 2 nodes:
- `kmeans_cluster` is a spark component which will conduct kmeans algorithom and print results.
- `show_output` is a command component which will show center points got from kmeans_cluster

In [ ]:
@pipeline()
def spark_pipeline_from_yaml(train_data):
    kmeans_clustering = spark_kmeans(file_input=train_data)
    kmeans_clustering.resources = {
        "instance_type": "Standard_E8S_V3",
        "runtime_version": "3.2.0",
    }
    kmeans_clustering.outputs.output.mode = InputOutputModes.DIRECT

    show_output = show_output_component(spark_output=kmeans_clustering.outputs.output)
    show_output.compute = cluster_name


sample_data = Input(
    path=parent_dir + "/data/sample_kmeans_data.txt",
    type=AssetTypes.URI_FOLDER,
    mode=InputOutputModes.DIRECT,
)

pipeline_job = spark_pipeline_from_yaml(train_data=sample_data)

In [ ]:
print(pipeline_job)

# 4. Submit pipeline job

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)